In [12]:
import selenium
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm
import pandas as pd


primary_url= 'https://m.bobaedream.co.kr/mycar/d_search_list/1?sel_m_gubun=&gubun=A&maker_no=&model_no=&group_no=&buy_year1=&buy_month1=&buy_year2=&buy_month2=&car_color=&car_option=&price1=100&price2=&min_km=&max_km=&chk-min-years=on&chk-min-month=on&chk-max-years=on&chk-max-month=on&lease_mprice1=&lease_mprice2=&lease_rmonth1=&lease_rmonth2=&lease_tprice1=&lease_tprice2=&l_year1=&l_year2='
resp = requests.get(primary_url)
soup = BeautifulSoup(resp.content)

# 마지막 페이지 가져오기 
a_tags = soup.select('a.last')
mlist = a_tags[0]['href'].split('/')
temp = mlist[3].split('?')
last_page = int(temp[0])

accidentKeys = ['total_loss', 'robbery','submerged','special_use', 'my_damage','opponent_damage','owner_changes','number_changes', 'rental_use','business_use','official_use']
repairKeys = ['도장', '탈착', '교환','판금','수리','기타']
total_columns = accidentKeys.extend(repairKeys)

df = pd.DataFrame(columns=total_columns)

# linkList = 차 하나 페이지 링크 리스트
linkList = []
url = 'https://m.bobaedream.co.kr/'
for pageNumber in tqdm(range(1,last_page+1)) :
    base_url = f'https://m.bobaedream.co.kr/mycar/d_search_list/{pageNumber}?sel_m_gubun=&gubun=A&maker_no=&model_no=&group_no=&buy_year1=&buy_month1=&buy_year2=&buy_month2=&car_color=&car_option=&price1=100&price2=&min_km=&max_km=&chk-min-years=on&chk-min-month=on&chk-max-years=on&chk-max-month=on&lease_mprice1=&lease_mprice2=&lease_rmonth1=&lease_rmonth2=&lease_tprice1=&lease_tprice2=&l_year1=&l_year2='
    resp = requests.get(base_url)
    soup = BeautifulSoup(resp.content)

    a_tags = soup.select('div.car-description > a.link')
    
    for a_tag in a_tags :
        linkList.append(a_tag['href'])


carNumber_cno_list = []
for link in tqdm(linkList) : 
    resp = requests.get(url+link)
    soup = BeautifulSoup(resp.content)

    try : 
        car_number = soup.select('h1.tit-area')[0].text
    except : 
        print('페이지 없음')
        continue
    cno = link.split('/')[3]
    carNumber_cno_list.append((car_number,cno))


for tup in tqdm(carNumber_cno_list) :
    # 차 하나 보험이력 페이지 
    history_url = f'https://m.bobaedream.co.kr/proc/mycarChart_B.php?car_number={tup[0]}&tbl=mycar&cno={tup[1]}'
    
    resp = requests.get(history_url)
    soup = BeautifulSoup(resp.content)


    div_tags = soup.select('div.price')
    
    accidentDict = {}
    if len(div_tags) != 0 :
        accidentValues = []
        for div in div_tags[:11] :
            value = div.text.strip('\n\t ') 
            #value = 0 if value == '없음' else value.replace('회','').replace('(','').replace('원)','').replace(',','')
            accidentValues.append(value)
        for k, v in zip(accidentKeys,accidentValues) :
            accidentDict[k] = v

        # if accidentDict['my_damage'] != 0 :
        #     my_damage = accidentDict['my_damage'].split(' ')
        #     accidentDict['my_damage_cnt'] = my_damage[0]
        #     accidentDict['my_damage_price'] = my_damage[1]
        # else : 
        #     accidentDict['my_damage_cnt'] = 0
        #     accidentDict['my_damage_price'] = 0

        # if accidentDict['opponent_damage'] != 0 :
        #     opponent_damage = accidentDict['opponent_damage'].split(' ')
        #     accidentDict['opponent_damage_cnt_damage_cnt'] = opponent_damage[0]
        #     accidentDict['opponent_damage_price'] = opponent_damage[1]
        # else : 
        #     accidentDict['opponent_damage_cnt_damage_cnt'] = 0
        #     accidentDict['opponent_damage_price'] = 0

        # accidentDict.pop('my_damage')
        # accidentDict.pop('opponent_damage')
    else :
        for k in accidentKeys :
            accidentDict[k] = None

    repairDict = {}
    
    repairValues = []
    div_tags = soup.select('div#repairAll div.td')
    if len(div_tags)==0 :
        for k in repairKeys :
            repairDict[k] = None
    else :
        cntIdx = 2
        for div in div_tags :
            repairValues.append(int(div.text.split(' ')[cntIdx]))
        for idx in range(len(repairKeys)) :
            repairDict[repairKeys[idx]] = repairValues[idx]

    total_dict = {}
    total_dict['car_number'] = tup[0]
    total_dict['cno'] = tup[1]
    for accident_key in accidentDict.keys() :
        total_dict[accident_key] = accidentDict[accident_key]
    for repair_key in repairDict.keys() :
        total_dict[repair_key] = repairDict[repair_key]

    temp_df = pd.DataFrame(total_dict, index = [0])
    df = pd.concat([df, temp_df], ignore_index=True)


      

  0%|          | 0/47 [00:00<?, ?it/s]

  0%|          | 0/2331 [00:00<?, ?it/s]

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

페이지 없음
페이지 없음
페이지 없음


  0%|          | 0/2328 [00:00<?, ?it/s]

In [13]:
df

,car_number,cno,total_loss,robbery,submerged,special_use,my_damage,opponent_damage,owner_changes,number_changes,rental_use,business_use,official_use,도장,탈착,교환,판금,수리,기타
0,224호1907,2231468,없음,없음,없음,있음,없음,없음,없음,없음,없음,없음,없음,None,None,None,None,None,None
1,200호2426,2220495,없음,없음,없음,있음,없음,없음,없음,없음,있음,없음,없음,None,None,None,None,None,None
2,201호3920,2224830,없음,없음,없음,있음,없음,없음,없음,없음,있음,없음,없음,None,None,None,None,None,None
3,200호8957,2222060,없음,없음,없음,있음,없음,없음,없음,없음,있음,없음,없음,None,None,None,None,None,None
4,201호8070,2225827,없음,없음,없음,있음,없음,없음,없음,없음,있음,없음,없음,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,233오4642,2209805,없음,없음,없음,없음,없음,없음,없음,없음,없음,없음,없음,None,None,None,None,None,None
2324,49오9576,2209804,없음,없음,없음,없음,없음,없음,없음,없음,없음,없음,없음,None,None,None,None,None,None
2325,34너2236,2209497,없음,없음,없음,없음,없음,없음,없음,없음,없음,없음,없음,None,None,None,None,None,None
2326,46구3338,2206650,없음,없음,없음,없음,없음,없음,없음,없음,없음,없음,없음,None,None,None,None,None,None


In [14]:
df.to_csv('insurance_history', index=False, encoding='utf-8')

In [53]:
df.drop(df[df['total loss'] == ''].index, inplace=True)

In [64]:
print(df['total loss'].unique())
print(df['robbery'].unique())

# df[df['submerged']=='있음']= True
# df[df['special_use']=='있음']= True
# df[df['rental_use']=='있음'] = True
# df[df['business_use']=='있음'] = True
# df[df['official_use']=='있음'] = True

# df[df['submerged']==0] = False
# df[df['special_use']==0] = False
# df[df['rental_use']==0] = False
# df[df['business_use']==0] = False
# df[df['official_use']==0] = False

[0]
[0]


C:\Users\user\AppData\Local\Temp\ipykernel_12688\3285545782.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'True' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df[df['special_use']=='있음']= True
C:\Users\user\AppData\Local\Temp\ipykernel_12688\3285545782.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'True' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df[df['special_use']=='있음']= True
C:\Users\user\AppData\Local\Temp\ipykernel_12688\3285545782.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'True' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df[df['special_use']=='있음']= True


In [10]:
categorical_features = df.select_dtypes(include=['object']).columns

for feature in categorical_features :
    print(f"{feature}의 고유값 개수 : {df[feature].nunique()}")

    print(df[feature].value_counts())
    print('\n')

car_number의 고유값 개수 : 50
car_number
14소0464     47
191고4038    47
58로5515     47
35노6460     47
69너7438     47
65어5569     47
08소0839     47
23소4258     47
55주8482     47
201소4241    47
62수5938     47
66무5735     47
25더1356     47
11소9507     47
26소1121     47
29보6719     47
46고0866     47
20주2090     47
64구7516     47
25버6566     47
22오2081     47
71노5059     47
48마6764     47
38보5773     47
17노2455     47
97소5633     47
287가8193    47
01루0226     47
03다0447     47
20머1006     47
83누8035     47
48저4960     47
844나3071    47
56고2198     47
34너0186     47
183우2283    47
191오2205    47
382무7000    47
64가1094     47
25서9475     47
53무8246     47
34주8439     47
816라5648    47
12서8279     47
02수9092     47
03서8446     47
69어6676     47
326구3714    47
25루1127     47
334두2926    47
Name: count, dtype: int64


cno의 고유값 개수 : 50
cno
2232423    47
2232745    47
2230541    47
2230547    47
2232776    47
2232778    47
2232862    47
2232711    47
2232709    47
2232746    47
2232736    47
2232773    4